## Transition indicators (TIs)

The present notebook aims to provide exemplary computation of TIs. Therefore, two systems are studied:
1. a linear system (therefore incapable of undergoing critical transitions) to test the presence of false positives.
2. a nonlinear system representing a 1D double-well that undergoes a critical transition to test true positives.

First we load the packages and self-written functions:

In [23]:
using DrWatson
@quickactivate "StatisticalEWS"

using LinearAlgebra, BenchmarkTools, RollingFunctions
using Statistics, StatsBase, Random, Distributions
using CairoMakie, DifferentialEquations, FFTW, CUDA
using SparseArrays

include(srcdir("utils.jl"))
include(srcdir("transition_indicators.jl"))
include(srcdir("significance_test.jl"))

check_std_endpoint (generic function with 1 method)

#### Model definition

We let both aforementioned models be forced by a user-defined function. In the present case, this is a simple linear drift such that the double-well model is driven out of its original equilibrium (i.e. experiences a transition).

In [24]:
# Deterministic part of linear system.
function f_linear(dx, x, p, t)
    dx[1] = p["λ"] * (x[1] + 1) + forcing(p, t)
end

# Deterministic part of double-well system.
function f_doublewell(dx, x, p, t)
    dx[1] = -x[1]^3 + x[1] + forcing(p, t)
end

# Stochastic part of system for noise.
function g_whitenoise(dx, x, p, t)
    dx[1] = p["σ"]
end

function forcing(p, t)
    return p["α"] * t
end

forcing (generic function with 1 method)

#### Perform simulation

In [25]:
# Define time step, vector and span. IC set for equilibrium of both systems.
dt = 1f-2
t = collect(0f0:dt:10f0)
tspan = extrema(t)
x0 = [-1f0]

pmodel = Dict("σ" => .1f0, "λ" => -1f0, "α" => .1f0)
models = [f_linear, f_doublewell]
labels = ["linear", "doublewell"]
nmodels = length(models)
X = zeros(Float32, nmodels, length(t))

for (f, lbl, i) in zip(models, labels, 1:nmodels)
    prob = SDEProblem(f, g_whitenoise, x0, tspan, pmodel)
    sol = solve(prob, EM(), dt=dt)
    X[i, :] = vcat(sol.u...)
end

In [44]:
nrows, ncols = 3, nmodels
fig = Figure( resolution = (1000, 1500) )
axs = [[Axis(
    fig[i,j],
    title = ( i==1 ? labels[j] : " "),
    xlabel = ( i==nrows ? "Time" : " "),
    ) for j in 1:ncols] for i in 1:nrows]

[lines!(axs[1][i], t, X[i, :], label = "data") for i in 1:ncols]
[ylims!(axs[1][i], (-1.2, 1.5)) for i in 1:ncols]
fig

Scene (1000px, 1500px):
  0 Plots
  6 Child Scenes:
    ├ Scene (1000px, 1500px)
    ├ Scene (1000px, 1500px)
    ├ Scene (1000px, 1500px)
    ├ Scene (1000px, 1500px)
    ├ Scene (1000px, 1500px)
    └ Scene (1000px, 1500px)

#### Detrend time series

In order to obtain the noise response, the time series is detrended. For this, several algorithms can be used:
- Rolling average
- Gaussian kernel
- Butterworth filter
- More to come...

In [27]:
T_smooth_wndw = 1f-1          # half-width for smoothing window .
T_indctr_wndw = 5f-1          # half-width for computation of indicator.
T_indctr_strd = 5f-1          # stride for computation of indicator.
T_signif_wndw = 1f0           # half-width for computation of indicator significance.
T_signif_strd = 1f0           # stride for computation of indicator significance.
T = [dt, T_smooth_wndw, T_indctr_wndw, T_indctr_strd, T_signif_wndw, T_signif_strd]
pwin = get_windowing_params(T)
window = centered_window
trend = gettrend_rollmean

Xtrend = mapslices( x_ -> trend(x_, pwin.N_smooth_wndw), X; dims=2 )
ttrend = trim_win( t, pwin.N_smooth_wndw )
[lines!(axs[1][i], ttrend, Xtrend[i, :], label = "trend") for i in 1:nmodels]
fig

MethodError: MethodError: no method matching WindowingParams(::Float32, ::Float32, ::Float32, ::Float32, ::Float32, ::Int64, ::Int64, ::Int64, ::Int64)
Closest candidates are:
  WindowingParams(::Real, ::Real, ::Real, ::Real, ::Real, ::Real, ::Int64, ::Int64, ::Int64, !Matched::Int64, !Matched::Int64) at ~/pCloudSync/PhD/Projects/StatisticalEWS/src/utils.jl:5
  WindowingParams(::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, !Matched::Any, !Matched::Any) at ~/pCloudSync/PhD/Projects/StatisticalEWS/src/utils.jl:5

#### Compute exemplary TIs

The most commonly used TIs are the variance and the coefficient of an AR1 regression of the residual. They are both expected to increase whenever a critical transition is near. Let's compute them and see what happens!

In [28]:
Xres_cpu = trim_win(X, pwin.N_smooth_wndw) - Xtrend
var_cpu = slide_estimator( Xres_cpu, pwin.N_indctr_wndw, variance )
ar1_cpu = slide_estimator( Xres_cpu, pwin.N_indctr_wndw, ar1_whitenoise )
tindctr = trim_win(ttrend, pwin.N_indctr_wndw)

[lines!(axs[2][j], tindctr, ar1_cpu[j, :], label = "ar1_cpu") for j in 1:ncols]
[ylims!(axs[2][i], (-.1, 1)) for i in 1:ncols]
[lines!(axs[3][j], tindctr, var_cpu[j, :], label = "var_cpu") for j in 1:ncols]
fig

UndefVarError: UndefVarError: pwin not defined

## Statistical significance

As we can observe in the previous plot, the critical transition of the double-well system in fact leads to an increase of the TIs!.. But such an increase also happens on the linear system, where no critical transition is possible. Meh. This means that along with true positives, we also want to get false ones.

Our way out of producing false positives is to test statistical significance by:
- Generating Fourier surrogates of the time series.
- Compute the transition indicator on each of these surrogates.
- Compute a metric that renders an increase of this indicator.
- Compare the increase metric of our actual time series with the ones computed on the surrogates.

The last step allows us to better investigate whether the increase of the TI is an artefact or not.

#### Fourier surrogates

Getting more data can be expensive (e.g. run a large model) or even barely possible (e.g. gather real-world data of the Earth system). However, (Fourier) surrogates of a given time series can be generated by:
- computing the Fourier transform of the time series.
- apply a random phase shift to each frequency.
- compute the inverse transform of the shifted spectrum.

To understand this with a simple example, one can imagine to have a time series x(t) given by:

$$ x(t) = \sin(t) + \sin(2 \pi \, t + \pi) $$

A Fourier surrogate of this could be:

$$ x_s(t) = \sin(t + 0.4 \, \pi) + \sin(2 \pi \, t + 2.7 \, \pi) $$

To generate such surrogates, TransitionIndicators.jl provides a ready-to-use function:

In [29]:
ns = 10000
Scpu = surrogates2D( fourier_surrogates2D(Xres_cpu, ns)... )

UndefVarError: UndefVarError: Xres_cpu not defined

#### Compute surrogate TIs

We can now perform the computation of the TIs by simply using:

In [30]:
STI = slide_estimator( Scpu.S, pwin.N_indctr_wndw, ar1_whitenoise )

UndefVarError: UndefVarError: Scpu not defined

Ouch. 10 000 surrogates is quite substantial and implies large dimension of the 3D surrogate array which in turn (surprise, surprise) leads to large computational cost. Moreover, the present case handles only two time series but applications sometimes present thousands of them (e.g. resulting from a 100 x 100 2D field). We might need something that scales better!

Instead of performing loops, we can transform the whole computation into a sparse array multiplication, which saves a lot of time:

In [31]:
STIsp = ar1_whitenoise( Scpu.S, pwin )[:, pwin.N_indctr_wndw+1:end-pwin.N_indctr_wndw+1]

UndefVarError: UndefVarError: Scpu not defined

#### GPU accelerated TI computation

Some operations can be efficiently parallelised on GPUs. For high dimensionality, this can provide a significant speed-up, as e.g.:

In [32]:
ntest = 1000
Acpu = rand(ntest, ntest)
@benchmark $Acpu * $Acpu

BenchmarkTools.Trial: 336 samples with 1 evaluation.
 Range (min … max):   8.613 ms … 24.235 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.350 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.866 ms ±  1.951 ms  ┊ GC (mean ± σ):  0.49% ± 2.29%

                       ▃▇█▆▆▅▂▁ ▂                              
  ▄▆▄▁▁▁▁▁▁▁▄▆▁▁▄▁▄▆▄▄▆████████▇█▇▆▁▆▇▇▆▇▇▇▁▁▇▄▄▁▆▁▇▆▇▁▁▁▄▄▁▄ ▇
  8.61 ms      Histogram: log(frequency) by time      22.5 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [33]:
Agpu = CUDA.rand(ntest, ntest)
@benchmark $Agpu * $Agpu

OutOfGPUMemoryError: Out of GPU memory trying to allocate 3.815 MiB
Effective GPU memory usage: 99.81% (7.780 GiB/7.795 GiB)
Memory pool usage: 6.687 GiB (6.688 GiB reserved)

For some TIs, a GPU-accelerated version of the computation has been implemented. This is the case for:
- Variance
- Skewness
- Kurtosis
- AR1 coefficient with white noise assumption

In [34]:
Sgpu = CuArray(Scpu.S)

UndefVarError: UndefVarError: Scpu not defined

In [35]:
M = gpuMask(Sgpu, pwin)

UndefVarError: UndefVarError: Sgpu not defined

In [36]:
STIgpu = ar1_whitenoise(Sgpu, M)

UndefVarError: UndefVarError: Sgpu not defined

In [37]:
STIgpucpu = Array(STIgpu)[:, pwin.N_indctr_wndw+1:end-pwin.N_indctr_wndw+1]

UndefVarError: UndefVarError: STIgpu not defined

In [38]:
i = 2000
fig_test = Figure()
ax_test = Axis(fig_test[1,1])
lines!(ax_test, STI[i, :])
lines!(ax_test, STIsp[i, :])
lines!(ax_test, STIgpucpu[i, :], linestyle = :dash)
xlims!(ax_test, (400, 600))
fig_test

UndefVarError: UndefVarError: STI not defined

In [39]:
sum( abs.(STIgpucpu - STI) .> 1f-1 ) # / length(STI)

UndefVarError: UndefVarError: STIgpucpu not defined

Let's benchmark our solutions, going from worst to best!

In [40]:
@benchmark slide_estimator( $Scpu.S, $pwin.N_indctr_wndw, ar1_whitenoise )

UndefVarError: UndefVarError: Scpu not defined

In [41]:
@benchmark ar1_whitenoise( $Scpu.S, $pwin )

UndefVarError: UndefVarError: Scpu not defined

In [42]:
@benchmark ar1_whitenoise( $Sgpu, $M )

UndefVarError: UndefVarError: Sgpu not defined

On the machine where the code was tested, the CPU function needs about 0.7 seconds to run, whereas the GPU only needs about 0.02 seconds. This leads to a roughly estimated speed-up of factor 35 - take that computational complexity! The absolute values are of course machine-specific and you might observe different ones.

In [43]:
B = CuArray( reshape(1f0:980f0*2f0, 2, 980) )
cumean(B, M, pwin)

UndefVarError: UndefVarError: M not defined